In [ ]:
import dspy 
import duckdb 
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv

load_dotenv()




In [32]:
dspy.configure(lm= dspy.LM('openai/gpt-4o-mini', max_tokens =800, api_key=os.getenv('OPENAI_API_KEY')))


print(dspy.settings.lm)

In [ ]:

excel_path = 'My Timesheet 24th Feb.xlsx'

sheet_names = pd.ExcelFile(excel_path).sheet_names

conn = duckdb.connect()


# Create tables for each sheet
for sheet in sheet_names:
    df = pd.read_excel(excel_path, sheet_name=sheet)
    # Register each DataFrame as a table named after the sheet
    conn.register(sheet, df)


# Show all tables in DuckDB
tables = conn.execute("SHOW TABLES").fetchall()
print("Tables in DuckDB:")
for table in tables:
    # Get the first few rows of each table to show structure
    try:
        head_data = conn.execute(f"SELECT * FROM {table[0]} LIMIT 3").df().to_markdown()
        display(head_data)

    except Exception as e:
        print(f"  Error fetching head of {table[0]}: {e}")
    print()  # Add blank line for readability




# Read Excel file using DuckDB
# excel = duckdb.sql("SELECT * FROM read_excel('My Timesheet 24th Feb.xlsx')")



# help(excel)





# Preprocessing steps
# 1. Drop empty rows and columns
# excel_df.dropna(how='all', inplace=True)  # Remove empty rows
# excel_df.dropna(how='all', axis=1, inplace=True)  # Remove empty columns

# # 2. Clean column names
# excel_df.columns = excel_df.columns.str.strip()  # Remove extra spaces

# # 3. Convert Excel data to CSV with UTF-8-sig encoding
# csv_buffer = io.StringIO()
# excel_df.to_csv(csv_buffer, index=False, encoding='utf-8-sig')
# csv_buffer.seek(0)

# # Read the processed CSV back into a dataframe
# new_df = pd.read_csv(csv_buffer)


# excel

table

Tables in DuckDB:
- Month1


'|    | Month Totals   |   Xperra |   SSC  |   Michael |   Zac |   Total |   Unnamed: 6 |   Unnamed: 7 |\n|---:|:---------------|---------:|-------:|----------:|------:|--------:|-------------:|-------------:|\n|  0 | Week1          |      900 |      0 |         0 |     0 |     900 |          nan |          nan |\n|  1 | Week2          |      900 |      0 |         0 |     0 |     900 |          nan |          nan |\n|  2 | Week3          |     1200 |      0 |         0 |     0 |    1200 |          nan |          nan |'


- Week1


'|    | Week/Start Min   | Unnamed: 1   | Start    | 2025-02-24 00:00:00   | End   | 2025-03-03 00:00:00   | Unnamed: 6   | Unnamed: 7   | Unnamed: 8   | Unnamed: 9   | Unnamed: 10   |\n|---:|:-----------------|:-------------|:---------|:----------------------|:------|:----------------------|:-------------|:-------------|:-------------|:-------------|:--------------|\n|  0 | Start            | End          | MON      | TUE                   | WED   | THU                   | FRI          | SAT          | SUN          |              |               |\n|  1 | 00:00:00         | 00:30:00     |          |                       |       |                       |              |              |              |              |               |\n|  2 | 00:30:00         | 01:00:00     |          |                       |       |                       |              |              |              |              |               |'


- Week2


...


- Week3


'|    | Week/Start Min   | Unnamed: 1   | Start    | 2025-03-10 00:00:00   | End   | 2025-03-17 00:00:00   | Unnamed: 6   | Unnamed: 7   | Unnamed: 8   | Unnamed: 9   | Unnamed: 10   |\n|---:|:-----------------|:-------------|:---------|:----------------------|:------|:----------------------|:-------------|:-------------|:-------------|:-------------|:--------------|\n|  0 | Start            | End          | MON      | TUE                   | WED   | THU                   | FRI          | SAT          | SUN          |              |               |\n|  1 | 00:00:00         | 00:30:00     |          |                       |       |                       |              |              |              |              |               |\n|  2 | 00:30:00         | 01:00:00     |          |                       |       |                       |              |              |              |              |               |'


- Week4


'|    | Week/Start Min   | Unnamed: 1   | Start    | 2025-03-17 00:00:00   | End   | 2025-03-24 00:00:00   | Unnamed: 6   | Unnamed: 7   | Unnamed: 8   | Unnamed: 9   | Unnamed: 10   |\n|---:|:-----------------|:-------------|:---------|:----------------------|:------|:----------------------|:-------------|:-------------|:-------------|:-------------|:--------------|\n|  0 | Start            | End          | MON      | TUE                   | WED   | THU                   | FRI          | SAT          | SUN          |              |               |\n|  1 | 00:00:00         | 00:30:00     |          |                       |       |                       |              |              |              |              |               |\n|  2 | 00:30:00         | 01:00:00     |          |                       |       |                       |              |              |              |              |               |'

('Week4',)

In [61]:
class data_context_gen(dspy.Signature):
    """
    Generate a compact JSON data context for DuckDB tables ingested from Excel or CSV files.
    The JSON must include:
    - Exact DuckDB table names
    - Source sheet or file name for each table
    - Table role (fact/dimension)
    - Primary key (pk)
    - Columns with type and role (pk, fk, attr, cat, measure, temporal)
    - Relationships between tables (foreign keys), with cardinality types (1:1, 1:M, M:1, M:M)
    - Business purpose of each table
    - Metrics expressed as formulas
    - Use cases for the dataset

    Example JSON format:
    {
      "tables": {
        "customer_master": {
          "source": "Customer_Master sheet",
          "role": "dimension",
          "pk": "customer_id",
          "columns": {
            "customer_id": {"type": "string", "role": "pk"},
            "name": {"type": "string", "role": "attr"},
            "region": {"type": "string", "role": "cat"},
            "signup_date": {"type": "date", "role": "temporal"}
          },
          "purpose": "Customer attributes for segmentation"
        },
        "sales_data": {
          "source": "Sales_Data sheet",
          "role": "fact",
          "pk": "order_id",
          "columns": {
            "order_id": {"type": "string", "role": "pk"},
            "customer_id": {"type": "string", "role": "fk"},
            "product_id": {"type": "string", "role": "fk"},
            "order_date": {"type": "date", "role": "temporal"},
            "quantity": {"type": "int", "role": "measure"},
            "unit_price": {"type": "decimal", "role": "measure"}
          },
          "purpose": "Transaction records for revenue analysis"
        },
        "product_catalog": {
          "source": "Product_Catalog sheet",
          "role": "dimension",
          "pk": "product_id",
          "columns": {
            "product_id": {"type": "string", "role": "pk"},
            "product_name": {"type": "string", "role": "attr"},
            "category": {"type": "string", "role": "cat"},
            "subcategory": {"type": "string", "role": "cat"},
            "brand": {"type": "string", "role": "cat"}
          },
          "purpose": "Product hierarchy for analysis"
        }
      },
      "relationships": [
        {"from": "sales_data.customer_id", "to": "customer_master.customer_id", "type": "M:1"},
        {"from": "sales_data.product_id", "to": "product_catalog.product_id", "type": "M:1"}
      ],
      "metrics": [
        "revenue = quantity * unit_price",
        "customer_lifetime_value"
      ],
      "use_cases": [
        "cohort analysis",
        "product performance",
        "regional sales"
      ]
    }

    Column roles: pk (primary key), fk (foreign key), attr (attribute), cat (categorical), measure (numerical), temporal (date/time)
    Table roles: fact (transactional), dimension (reference data)
    Relationship types: 1:1, 1:M, M:1, M:M
    """
    user_description = dspy.InputField(desc="User's description of the data, including relationships")
    dataset_view = dspy.InputField(desc="Dataset name with sample head(5 rows) view")
    data_context = dspy.OutputField(desc="Compact JSON describing DuckDB tables, columns, relationships, metrics and use cases")

In [71]:
user_description = "These are my worksheets month over month"

data_context_agent = dspy.Predict(data_context_gen)

tables = conn.execute("SHOW TABLES").fetchall()

dataset_view = ""

for table in tables:
    head_data = conn.execute(f"SELECT * FROM {table[0]} LIMIT 3").df().to_markdown()

    dataset_view+="exact_table_name="+table[0]+'\n:'+head_data+'\n'


response = data_context_agent(user_description=user_description, dataset_view=dataset_view)


display(response)



Prediction(
    data_context='{\n  "tables": {\n    "Month1": {\n      "source": "Month1 sheet",\n      "role": "fact",\n      "pk": "week",\n      "columns": {\n        "week": {"type": "string", "role": "pk"},\n        "month_totals": {"type": "int", "role": "measure"},\n        "xperra": {"type": "int", "role": "measure"},\n        "ssc": {"type": "int", "role": "measure"},\n        "michael": {"type": "int", "role": "measure"},\n        "zac": {"type": "int", "role": "measure"},\n        "total": {"type": "int", "role": "measure"}\n      },\n      "purpose": "Monthly totals for performance tracking"\n    },\n    "Week1": {\n      "source": "Week1 sheet",\n      "role": "dimension",\n      "pk": "week_start",\n      "columns": {\n        "week_start": {"type": "datetime", "role": "pk"},\n        "start": {"type": "time", "role": "temporal"},\n        "end": {"type": "time", "role": "temporal"}\n      },\n      "purpose": "Details of the first week for time analysis"\n    },\n    "We

In [72]:
print(response.data_context)

{
  "tables": {
    "Month1": {
      "source": "Month1 sheet",
      "role": "fact",
      "pk": "week",
      "columns": {
        "week": {"type": "string", "role": "pk"},
        "month_totals": {"type": "int", "role": "measure"},
        "xperra": {"type": "int", "role": "measure"},
        "ssc": {"type": "int", "role": "measure"},
        "michael": {"type": "int", "role": "measure"},
        "zac": {"type": "int", "role": "measure"},
        "total": {"type": "int", "role": "measure"}
      },
      "purpose": "Monthly totals for performance tracking"
    },
    "Week1": {
      "source": "Week1 sheet",
      "role": "dimension",
      "pk": "week_start",
      "columns": {
        "week_start": {"type": "datetime", "role": "pk"},
        "start": {"type": "time", "role": "temporal"},
        "end": {"type": "time", "role": "temporal"}
      },
      "purpose": "Details of the first week for time analysis"
    },
    "Week2": {
      "source": "Week2 sheet",
      "role": "dime

In [42]:
print(dataset_view)


Month1
:|    | Month Totals   |   Xperra |   SSC  |   Michael |   Zac |   Total |   Unnamed: 6 |   Unnamed: 7 |
|---:|:---------------|---------:|-------:|----------:|------:|--------:|-------------:|-------------:|
|  0 | Week1          |      900 |      0 |         0 |     0 |     900 |          nan |          nan |
|  1 | Week2          |      900 |      0 |         0 |     0 |     900 |          nan |          nan |
|  2 | Week3          |     1200 |      0 |         0 |     0 |    1200 |          nan |          nan |
Week1
:|    | Week/Start Min   | Unnamed: 1   | Start    | 2025-02-24 00:00:00   | End   | 2025-03-03 00:00:00   | Unnamed: 6   | Unnamed: 7   | Unnamed: 8   | Unnamed: 9   | Unnamed: 10   |
|---:|:-----------------|:-------------|:---------|:----------------------|:------|:----------------------|:-------------|:-------------|:-------------|:-------------|:--------------|
|  0 | Start            | End          | MON      | TUE                   | WED   | THU         

In [67]:
class data_maker(dspy.Signature):
    """
    Generate DuckDB SQL queries to fetch data from multiple datasets. Handle joins, aggregations, and filtering across tables.
    Use table names as they appear in dataset_descriptions. Common patterns:
    
    Single table: SELECT * FROM customer_master WHERE region = 'North'
    Join tables: SELECT c.name, SUM(s.quantity * s.unit_price) as revenue 
                 FROM customer_master c JOIN sales_data s ON c.customer_id = s.customer_id
    Multi-dataset: SELECT e.first_name, a.hours_worked FROM employee_info e 
                   JOIN attendance_log a ON e.emp_id = a.emp_id WHERE a.date = '2024-01-15'
    Aggregation: SELECT category, COUNT(*) as products FROM product_catalog GROUP BY category
    Time-based: SELECT DATE_TRUNC('month', order_date) as month, SUM(quantity) 
                FROM sales_data WHERE order_date >= '2024-01-01' GROUP BY month
    
    Always return: df = conn.execute('SQL_QUERY').df() or more
    """
    user_query = dspy.InputField(desc="what the user is requesting")
    dataset_descriptions = dspy.InputField(desc="Dict of dataset contexts with table names, columns, and relationships")
    duckdb_sql = dspy.OutputField(desc="df = conn.execute('SQL query to fetch the right data').df()")

In [68]:
data_maker_agent = dspy.Predict(data_maker)

user_query = "show me how much I made from Xperra"

dataset_descriptions = str(response.data_context)

sql = data_maker_agent(user_query=user_query, dataset_descriptions=dataset_descriptions)



In [69]:
exec(sql.duckdb_sql)

In [70]:
print(df)

   total_xperra
0       27240.0


In [60]:
conn.execute("SHOW TABLES").df()


,name
0,Month1
1,Week1
2,Week2
3,Week3
4,Week4
